In [1]:
!pip install transformers tensorflow
import transformers
import tensorflow as tf

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [55]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import tkinter as tk

num_labels = 2

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_carregado = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

criterion = torch.nn.CrossEntropyLoss()

if 'DISPLAY' in __import__('os').environ:
    janela = tk.Tk()
    janela.title("Chat com BERT")

    input_usuario = tk.Text(janela, height=10, width=40)
    input_usuario.pack()

    resposta_bert = tk.Text(janela, height=10, width=40)
    resposta_bert.pack()

    def enviar_input():
        input_texto = input_usuario.get("1.0", tk.END)
        inputs = tokenizer(input_texto, return_tensors='pt')
        outputs = model_carregado(inputs['input_ids'], attention_mask=inputs['attention_mask'])
        resposta = tokenizer.decode(outputs.logits.argmax(dim=-1))
        resposta_bert.insert(tk.END, resposta)

    botao_enviar = tk.Button(janela, text="Enviar", command=enviar_input)
    botao_enviar.pack()

    janela.mainloop()
else:
    print("Execução apenas em ambiente local com DISPLAY definido.")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Execução apenas em ambiente local com DISPLAY definido.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim


In [5]:
import pandas as pd

dados = {
    "texto": ["Olá, como você está?", "Estou bem, obrigado!", "Que bom!", "E você?", "Estou ótimo!"],
    "label": [0, 0, 0, 1, 1]
}

df = pd.DataFrame(dados)

In [6]:
from sklearn.model_selection import train_test_split

texto_treino, texto_teste, label_treino, label_teste = train_test_split(df['texto'], df['label'], random_state=42, test_size=0.2)


In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

inputs_treino = tokenizer(texto_treino.tolist(), return_tensors='pt', max_length=512, padding='max_length', truncation=True)
inputs_teste = tokenizer(texto_teste.tolist(), return_tensors='pt', max_length=512, padding='max_length', truncation=True)

labels_treino = torch.tensor(label_treino.tolist())
labels_teste = torch.tensor(label_teste.tolist())


In [56]:
from transformers import BertModel

modelo_bert = BertModel.from_pretrained('bert-base-uncased')

from torch.optim import Adam
import torch.nn as nn
import torch

otimizador = Adam(modelo_bert.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo_bert.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [57]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

modelo_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

texto_teste = ["Exemplo de texto para teste"]
labels_teste = torch.tensor([1])

inputs_teste = tokenizer(texto_teste, return_tensors='pt', max_length=512, padding='max_length', truncation=True)

input_ids = inputs_teste['input_ids']
attention_mask = inputs_teste['attention_mask']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo_bert = modelo_bert.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels_teste = labels_teste.to(device)

outputs = modelo_bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels_teste)

loss = outputs.loss
print(f"Loss: {loss.item()}")

logits = outputs.logits
_, predicted = torch.max(logits, dim=1)
acuracia = (predicted == labels_teste).sum().item() / len(labels_teste)
print(f"Acurácia: {acuracia:.2f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.6050856113433838
Acurácia: 1.00


In [12]:
input_ids = inputs_teste['input_ids'].to(device)
attention_mask = inputs_teste['attention_mask'].to(device)

In [13]:
modelo_bert.eval()
previsoes = modelo_bert(inputs_teste['input_ids'].to(device), attention_mask=inputs_teste['attention_mask'].to(device))
previsoes = torch.argmax(previsoes.logits, dim=1)



In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

acuracia = accuracy_score(labels_teste.cpu().numpy(), previsoes.cpu().numpy())
precisao = precision_score(labels_teste.cpu().numpy(), previsoes.cpu().numpy(), average='weighted')
revocacao = recall_score(labels_teste.cpu().numpy(), previsoes.cpu().numpy(), average='weighted')

print(f'Acurácia: {acuracia:.3f}, Precisão: {precisao:.3f}, Revocação: {revocacao:.3f}')



Acurácia: 1.000, Precisão: 1.000, Revocação: 1.000


In [15]:
pip install Flask torch transformers


In [16]:
pip install flask torch torchvision transformers numpy pandas


In [17]:
!mkdir templates


In [18]:
!touch app.py
!touch model.py
!touch templates/index.html

In [19]:
%%writefile app.py
from flask import Flask, request, render_template
from model import get_response

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        texto = request.form['texto']
        resposta = get_response(texto)
        return render_template('index.html', resposta=resposta)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)



Overwriting app.py


In [20]:
%%writefile model.py
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
modelo = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

def get_response(texto):
    inputs = tokenizer(texto, return_tensors='pt')
    outputs = modelo(**inputs)
    resposta = torch.argmax(outputs.logits, dim=1)
    return resposta.item()




Overwriting model.py


In [21]:
%%writefile templates/index.html
<html>
  <body>
    <h1>Teste do Modelo BERT</h1>
    <form method="post">
      <textarea name="texto"></textarea>
      <input type="submit" value="Enviar">
    </form>
    {% if resposta %}
      <p>Resposta: {{ resposta }}</p>
    {% endif %}
  </body>
</html>

Overwriting templates/index.html


In [22]:
!pip install flask-ngrok

In [58]:
def buscar_capital_terminal():

    capitais_brasileiras = {
        "Acre": "Rio Branco",
        "Alagoas": "Maceió",
        "Amapá": "Macapá",
        "Amazonas": "Manaus",
        "Bahia": "Salvador",
        "Ceará": "Fortaleza",
        "Distrito Federal": "Brasília",
        "Espírito Santo": "Vitória",
        "Goiás": "Goiânia",
        "Maranhão": "São Luís",
        "Mato Grosso": "Cuiabá",
        "Mato Grosso do Sul": "Campo Grande",
        "Minas Gerais": "Belo Horizonte",
        "Pará": "Belém",
        "Paraíba": "João Pessoa",
        "Paraná": "Curitiba",
        "Pernambuco": "Recife",
        "Piauí": "Teresina",
        "Rio de Janeiro": "Rio de Janeiro",
        "Rio Grande do Norte": "Natal",
        "Rio Grande do Sul": "Porto Alegre",
        "Rondônia": "Porto Velho",
        "Roraima": "Boa Vista",
        "Santa Catarina": "Florianópolis",
        "São Paulo": "São Paulo",
        "Sergipe": "Aracaju",
        "Tocantins": "Palmas"
    }

    estado = input("Digite o nome de um estado brasileiro: ").strip().title()
    if estado in capitais_brasileiras:
        print(f"A capital de {estado} é {capitais_brasileiras[estado]}.")
    else:
        print("Estado não encontrado. Tente novamente.")

buscar_capital_terminal()

Digite o nome de um estado brasileiro: Maranhão
A capital de Maranhão é São Luís.
